## SetUp

In [1]:
import boto3
import faiss
from langchain_aws import ChatBedrockConverse
from langchain_aws import BedrockEmbeddings
from langchain_community.docstore import InMemoryDocstore
from langchain_community.vectorstores import FAISS

In [ ]:
bedrock_client = boto3.client("bedrock-runtime", region_name="us-west-2")
model_id = "us.amazon.nova-lite-v1:0"
provider_id = "amazon"
embedding_id = "amazon.titan-embed-text-v2:0"

llm = ChatBedrockConverse(
    model=model_id,
    provider=provider_id,
    temperature=0.5,
    max_tokens=None,
    client=bedrock_client
)

embeddings = BedrockEmbeddings(
    model_id=embedding_id,
    client=bedrock_client
)

In [ ]:
embedding_dim = len(embeddings.embed_query("hello world"))
index = faiss.IndexFlatL2(embedding_dim)

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)